In [113]:
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import asyncio
import aiohttp
from aiohttp import ClientTimeout
import csv

In [114]:
# Các hàm xử lý phụ 

# Hàm xử lý ngày tháng năm
def parse_date(date_text):
    try:
        date_obj = datetime.strptime(date_text, '%d/%m/%Y')
        return date_obj.day, date_obj.month, date_obj.year
    except ValueError:
        print(f"Unable to parse date: {date_text}")
        return None, None, None
    
# Hàm để xử lý giá trị rỗng
def process_value(value, is_numeric=False):
    if value in [None, 'nan', 'NaN', 'N/A', '']:
        return 0 if is_numeric else "NOT FOUND"
    return value    

# Hàm chuyển đổi giá trị thành số
def convert_to_number(value):
    if isinstance(value, (int, float)):
        return value
    if isinstance(value, str):
        value = ''.join(filter(str.isdigit, value))
        return int(value) if value else 0
    return 0

In [115]:
# Hàm đăng nhập
def login_hako(username, password):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        driver.get('https://ln.hako.vn/login')
        
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "name"))
        )
        password_field = driver.find_element(By.ID, "password")
        
        username_field.clear()
        username_field.send_keys(username)
        password_field.clear()
        password_field.send_keys(password)
        
        input("Xử lý captcha, xong thì nhấn Enter sau khi chắc chắn đã đăng nhập thành công...")
        
        try:
            WebDriverWait(driver, 5).until(
                lambda driver: driver.current_url == "https://ln.hako.vn/"
            )
            print("Đăng nhập thành công!")
            return driver
        except:
            if "login" in driver.current_url:
                print("Đăng nhập thất bại! Vẫn ở trang login.")
            else:
                print("Đăng nhập thất bại! URL hiện tại:", driver.current_url)
            driver.quit()
            return None
            
    except Exception as e:
        print(f"Có lỗi: {str(e)}")
        driver.quit()
        return None

In [116]:
# Các hàm trích xuất thông tin

# Trích xuất ID
def extract_id_from_url(url):
    patterns = [
        r'/truyen/(\d+)',
        r'/sang-tac/(\d+)-',
        r'/convert/(\d+)-'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

# Trích xuất thông tin 
def extract_info_from_html(novel_id, html_content):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.title.string.strip() if soup.title else "NOT FOUND"
        title = re.sub(r'^\(\d+\)\s*', '', title)  
        title = re.sub(r' - Cổng Light Novel - Đọc Light Novel$', '', title) 
            
        # Trích xuất link
        canonical_link = soup.find('link', rel='canonical')
        canonical_url = canonical_link['href'] if canonical_link else "NOT FOUND"
        canonical_url = canonical_url.replace('docln.net', 'ln.hako.vn')
        url_id = extract_id_from_url(canonical_url)  

        # Trích xuất phương thức sáng tác
        method = "NOT FOUND"
        method_span = soup.find('div', class_='series-type')
        if method_span:
            method_link = method_span.find('span')
        if method_link and method_link.string:
            method = method_link.string.strip()

        # Trích xuất thông tin về thể loại
        genres = []
        manga = "Not sure"
        anime = "Not sure"
        cd = "Not sure"
        origin = "vietnamese" if method == "Truyện sáng tác" else "japanese"
        genre_items = soup.find_all(class_='series-gerne-item')
        for item in genre_items:
            genre_text = item.get_text(strip=True)
            if "Manga" in genre_text: manga = "Yes"
            if "Anime" in genre_text: anime = "Yes"
            if "CD" in genre_text: cd = "Yes"
            if "Chinese" in genre_text: origin = "chinese"
            if "English" in genre_text: origin = "english"
            if "Korean" in genre_text: origin = "korean"
            elif not any(word in genre_text for word in ["Manga", "Anime", "CD", "Chinese", "English", "Korean"]):
                genres.append(genre_text)
        genres = ", ".join(genres) if genres else "NOT FOUND"

        # Trích xuất link ảnh
        image_link = "NOT FOUND"
        content_div = soup.find('div', class_='content img-in-ratio')
        if content_div and 'style' in content_div.attrs:
            style = content_div['style']
            match = re.search(r"url\('([^']+)'\)", style)
            if match:
                image_link = match.group(1)
        
        # Trích xuất tác giả
        author = "NOT FOUND"
        author_span = soup.find('span', class_='info-name', string='Tác giả:')
        if author_span:
            info_value_span = author_span.find_next_sibling('span', class_='info-value')
            if info_value_span:
                author_link = info_value_span.find('a')
            if author_link:
                author = author_link.string.strip()
    
        # Trích xuất họa sĩ
        artist = "NOT FOUND"
        artist_span = soup.find('span', class_='info-name', string='Họa sĩ:')
        if artist_span:
            info_value_span = artist_span.find_next_sibling('span', class_='info-value')
            if info_value_span and info_value_span.string:
                artist = info_value_span.string.strip()

        # Trích xuất kiểu trình bày
        showtype = 'web novel' if method == "Truyện sáng tác" or artist.lower() in ['NOT FOUND', 'N/A'] else 'light novel'

        # Trích xuất tình trạng
        state = "NOT FOUND"
        state_span = soup.find('span', class_='info-name', string='Tình trạng:')
        if state_span:
            info_value_span = state_span.find_next_sibling('span', class_='info-value')
            if info_value_span:
                state_link = info_value_span.find('a')
                if state_link:
                    state = state_link.string.strip() 

        # Trích xuất số like
        like = 0
        like_span = soup.find('span', class_='block feature-value')
        if like_span:
            like_link = like_span.find_next_sibling('span', class_='block feature-name')
            if like_link and like_link.string:
                like_text = like_link.string.strip()
                try:
                    like = float(like_text)
                except ValueError:
                    pass

        # Trích xuất số từ
        nword = 0
        nword_span = soup.find('div', class_='statistic-name', string='Số từ')
        if nword_span:
            nword_link = nword_span.find_next_sibling('div', class_='statistic-value')
            if nword_link:
                nword = nword_link.string.strip()

        # Trích xuất số lượt đánh giá
        rate = 0  
        rate_span = soup.find('div', class_='statistic-name', string='Đánh giá')
        if rate_span:
            rate_link = rate_span.find_next_sibling('div', class_='statistic-value')
            if rate_link and rate_link.string:
                rate_text = rate_link.string.strip()
                try:
                    rate = float(rate_text)
                except ValueError:
                    pass

        # Trích xuất số lượt xem
        view = 0
        view_span = soup.find('div', class_='statistic-name', string='Lượt xem')
        if view_span:
            view_link = view_span.find_next_sibling('div', class_='statistic-value')
            if view_link:
                view = view_link.string.strip()

        # Trích xuất số lượt bình luận
        ncom = 0
        ncom_span = soup.find('span', class_='comments-count')
        if ncom_span:
            ncom = ncom_span.string.strip()
            ncom = re.sub(r'[()]', '', ncom)

        # Trích xuất tên gọi khác
        fname = None
        fname_span = soup.find('div', class_='fact-value')
        if fname_span:
            fname_links = fname_span.find_all('div', class_='block pad-bottom-5')
            if fname_links:
                fname_list = [link.get_text(strip=True) for link in fname_links if link.get_text(strip=True)]
                fname = chr(10).join(fname_list) if fname_list else 'None'

        # Trích xuất chủ thầu
        trans = None
        id_o_l = None
        id_o = None
        trans_span = soup.find('span', class_='series-owner_name')
        if trans_span:
            trans = trans_span.string.strip()
            next_o = trans_span.find_next('a')
            if next_o and 'href' in next_o.attrs:
                id_o_l = next_o['href']
                if not id_o_l.startswith(('https://', 'http://')):
                    id_o_l = 'https://ln.hako.vn' + id_o_l
                id_o_l = id_o_l.replace('docln.net', 'ln.hako.vn')
                id_o = re.search(r'/(\d+)$', id_o_l)
                id_o = id_o.group(1) if id_o else None

        # Trích xuất nhóm dịch
        team = None
        id_t_l = None
        id_t = None
        team_span = soup.find('div', class_='fantrans-value')
        if team_span:
            team = team_span.string.strip()
            next_t = team_span.find_next('a')
            if next_t and 'href' in next_t.attrs:
                id_t_l = next_t['href']
                if not id_t_l.startswith(('https://', 'http://')):
                    id_t_l = 'https://ln.hako.vn' + id_t_l
                else:
                    id_t_l = id_t_l.replace('docln.net', 'ln.hako.vn')
                id_t = re.search(r'/nhom-dich/(\d+)', id_t_l)
                id_t = id_t.group(1) if id_t else None

        # Trích xuất những người tham gia (nếu có)
        atb = None
        id_j_l = None
        id_j = None  
        atb_span = soup.find('div', class_='series-owner_share')
        if atb_span:
            atb_links = atb_span.find_all('a', class_='ln_info-name')
            if atb_links:
                atb_list = [link.get_text(strip=True) for link in atb_links if link.get_text(strip=True)]
                atb = ", ".join(atb_list) if atb_list else 'None' + ","
                id_j_l_list = [link.get('href') for link in atb_links if link.get('href')]
                id_j_l_list = ['https://ln.hako.vn' + url if not url.startswith(('https://', 'http://')) else url.replace('docln.net', 'ln.hako.vn') for url in id_j_l_list]
                id_j_l = ", ".join(id_j_l_list) if id_j_l_list else 'None' + ","
                id_j_list = [re.search(r'/(\d+)$', url).group(1) for url in id_j_l_list if re.search(r'/(\d+)$', url)]
                id_j = ", ".join(id_j_list) if id_j_list else 'None' + ","

        # Trích xuất số tập
        def count_vol(soup):
            vol_spans = soup.find_all('span', class_='list_vol-title')
            return len(vol_spans)
        nvol = count_vol(soup)

        # Trích xuất số chương
        def count_chap(soup):
            chap_spans = soup.find_all('div', class_='chapter-name')
            return len(chap_spans)
        nchap = count_chap(soup)

        # Xác định hình thức dựa trên số tập và trạng thái
        format_type = "series" if nvol > 1 else (
            "Not sure" if nvol == 1 and state.lower() == "đang tiến hành" else "oneshot"
        )

        # Trích xuất thời gian bắt đầu 
        first_day, first_month, first_year = None, None, None
        earliest_date = None  # Chuyển khai báo ra ngoài
        chapter_time_divs = soup.find_all('div', class_='chapter-time')
        for time_div in chapter_time_divs:
            date_text = time_div.get_text(strip=True)
            try:
                current_date = datetime.strptime(date_text, '%d/%m/%Y')
                if earliest_date is None or current_date < earliest_date:
                    earliest_date = current_date
            except ValueError:
                continue
        if earliest_date:
            first_day = earliest_date.day
            first_month = earliest_date.month
            first_year = earliest_date.year
            
        # Trích xuất thời gian cập nhật mới nhất
        latest_day, latest_month, latest_year = None, None, None
        latest_span = soup.find('div', class_='statistic-name', string='Lần cuối')
        if latest_span:
            time_div = latest_span.find_next_sibling('div', class_='statistic-value')
            if time_div:
                time_tag = time_div.find('time', class_='timeago')
                if time_tag and 'title' in time_tag.attrs:
                    latest_date_text = time_tag['title'].split()[0]  
                    latest_day, latest_month, latest_year = parse_date(latest_date_text)

        return (url_id,title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
                author, artist, showtype, state, like, nword, rate, view, fname, id_o, 
                trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, format_type,
                first_day, first_month, first_year, latest_day, latest_month, latest_year, ncom)
    
    except Exception as e:
        print(f"Error extracting info for novel {novel_id}: {str(e)}")
        return None
    pass

In [117]:
# Hàm xử lý thông tin
def process_novel_info(info):
    if info is None:
        return None
        
    try:
        (url_id, title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
         author, artist, showtype, state, like, nword, rate, view, fname, id_o, 
         trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, format_type,
         first_day, first_month, first_year, latest_day, latest_month, latest_year, ncom) = info
        
        data = {
            'ID': process_value(url_id),
            'Tựa đề': process_value(title),
            'Link hako': process_value(canonical_url),
            'Phương thức sáng tác': process_value(method),
            'Thể loại': process_value(genres),
            'Manga': process_value(manga),
            'Anime': process_value(anime),
            'CD': process_value(cd),
            'Ngôn ngữ gốc': process_value(origin),
            'Link ảnh': process_value(image_link),
            'Tác giả': process_value(author),
            'Họa sĩ': process_value(artist),
            'Loại hình': process_value(showtype),
            'Tình trạng': process_value(state),
            'Số like': convert_to_number(process_value(like, is_numeric=True)),
            'Số từ': convert_to_number(process_value(nword, is_numeric=True)),
            'Số lượt đánh giá': convert_to_number(process_value(rate, is_numeric=True)),
            'Số lượt xem': convert_to_number(process_value(view, is_numeric=True)),
            'Số lượt bình luận': convert_to_number(process_value(ncom, is_numeric=True)),
            'Fname': process_value(fname),
            'ID chủ thầu': process_value(id_o),
            'Chủ thầu': process_value(trans),
            'Link chủ thầu': process_value(id_o_l),
            'ID nhóm thầu': process_value(id_t),
            'Nhóm thầu': process_value(team),
            'Link nhóm thầu': process_value(id_t_l),
            'ID người tham gia': process_value(id_j),
            'Người tham gia': process_value(atb),
            'Link người tham gia': process_value(id_j_l),
            'Số tập': process_value(nvol),
            'Số chương': process_value(nchap),
            'Hình thức': process_value(format_type),
            'Ngày bắt đầu': process_value(first_day),
            'Tháng bắt đầu': process_value(first_month),   
            'Năm bắt đầu': process_value(first_year),
            'Ngày cập nhật cuối': process_value(latest_day),
            'Tháng cập nhật cuối': process_value(latest_month),
            'Năm cập nhật cuối': process_value(latest_year)
        }
        return data
    except Exception as e:
        print(f"Lỗi khi xử lý thông tin: {str(e)}")
        return None

In [118]:
# Hàm lấy ID cuối cùng đã xử lý
def get_last_processed_id(filename='hako_data.csv'):
    """Lấy ID cuối cùng đã xử lý từ csv"""
    try:
        df = pd.read_csv(filename)
        if not df.empty:
            return df['ID'].max()
        return 0
    except (FileNotFoundError, pd.errors.EmptyDataError):
        return 0

In [119]:
# Hàm đọc thông tin
def get_existing_data(filename='hako_data.csv'):
    """Đọc dữ liệu từ csv (nếu có):"""
    try:
        df = pd.read_csv(filename, sep=';', quoting=csv.QUOTE_ALL)
        return df
    except FileNotFoundError:
        return pd.DataFrame()
    except pd.errors.EmptyDataError:
        return pd.DataFrame()

In [120]:
# Hàm tạo tên cột (file csv trống)
def get_column_names():
    """Trả về danh sách tên cột chuẩn"""
    return ['ID', 'Tựa đề', 'Link hako', 'Phương thức sáng tác', 'Thể loại', 
            'Manga', 'Anime', 'CD', 'Ngôn ngữ gốc', 'Link ảnh', 'Tác giả', 
            'Họa sĩ', 'Loại hình', 'Tình trạng', 'Số like', 'Số từ', 
            'Số lượt đánh giá', 'Số lượt xem', 'Số lượt bình luận', 'Fname',
            'ID chủ thầu', 'Chủ thầu', 'Link chủ thầu', 'ID nhóm thầu', 
            'Nhóm thầu', 'Link nhóm thầu', 'ID người tham gia', 'Người tham gia',
            'Link người tham gia', 'Số tập', 'Số chương', 'Hình thức',
            'Ngày bắt đầu', 'Tháng bắt đầu', 'Năm bắt đầu',
            'Ngày cập nhật cuối', 'Tháng cập nhật cuối', 'Năm cập nhật cuối']

In [121]:
# Hàm lưu thông tin vào csv
def save_to_csv(new_data, filename='hako_data.csv', check_summary=False):
    """Lưu dữ liệu vào csv"""
    new_df = pd.DataFrame(new_data)
    
    existing_df = get_existing_data(filename)
    
    if existing_df.empty:
        columns = get_column_names()
        new_df = new_df.reindex(columns=columns)
        new_df.to_csv(filename, index=False, sep=';', quoting=csv.QUOTE_ALL)

        if check_summary:
            print(f"Đã tạo file mới và lưu {len(new_df)} bản ghi")
        return

    existing_df['ID'] = pd.to_numeric(existing_df['ID'])
    new_df['ID'] = pd.to_numeric(new_df['ID'])
    
    duplicate_ids = set(existing_df['ID']).intersection(set(new_df['ID']))
    
    if duplicate_ids and check_summary:
        print(f"Đã tìm thấy {len(duplicate_ids)} ID trùng lặp, sẽ sớm cập nhật")
    
    if duplicate_ids:
        existing_df = existing_df[~existing_df['ID'].isin(duplicate_ids)]

    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    
    combined_df = combined_df.sort_values('ID').reset_index(drop=True)
    
    combined_df.to_csv(filename, index=False, sep=';', quoting=csv.QUOTE_ALL)
    
    if check_summary:
        print(f"Đã lưu tổng cộng {len(combined_df)} bản ghi")
        print(f"- Giữ nguyên: {len(existing_df)} bản ghi")
        print(f"- Cập nhật/Thêm mới: {len(new_df)} bản ghi")

In [122]:
# Hàm tìm ID còn thiếu
def get_missing_ids(start_id, end_id, filename='hako_data.csv'):
    """Tìm các ID còn thiếu trong khoảng: """
    try:
        df = pd.read_csv(filename, sep=';', quoting=csv.QUOTE_ALL)
        existing_ids = set(df['ID'].astype(int))
        all_ids = set(range(start_id, end_id + 1))
        missing_ids = all_ids - existing_ids
        return sorted(list(missing_ids))
    except (FileNotFoundError, pd.errors.EmptyDataError):
        return list(range(start_id, end_id + 1))

In [123]:
# Hàm lưu mấy cái ID không lấy được thông tin (có gì còn chạy lại thử)
def save_failed_ids(failed_ids, filename='hako_null.csv'):
   """Lưu các ID không lấy được vào csv"""
   try:
       try:
           with open(filename, 'r') as f:
               existing_content = f.read().strip()
               existing_content = existing_content.replace(',\n', '\n')
               existing_ids = [int(id) for id in existing_content.replace('\n', ',').split(',') if id.strip()]
       except FileNotFoundError:
           existing_ids = []
       
       all_ids = sorted(set(existing_ids + failed_ids))

       with open(filename, 'w') as f:
           for i in range(0, len(all_ids), 25):
               chunk = all_ids[i:i+25]
               f.write(','.join(map(str, chunk)) + ',\n')
               
   except Exception as e:
       print(f"Lỗi khi lưu ID thất bại: {str(e)}")

In [124]:
# Hàm để fetch thông tin truyện
async def fetch_novel_info(session, id, semaphore, delay=0.5):
    """Hàm async để fetch thông tin truyện"""
    async with semaphore:  
        try:
            await asyncio.sleep(delay)  
            url = f'https://ln.hako.vn/truyen/{id}-abcd'
            async with session.get(url) as response:
                if response.status == 200:
                    html_content = await response.text()
                    return id, html_content
                return id, None
        except Exception as e:
            print(f"Lỗi khi fetch ID {id}: {str(e)}")
            return id, None

In [125]:
# Hàm xử lý các ID theo batch
async def process_batch_async(ids):
    """Xử lý một batch các ID bằng async"""
    novel_data = []
    failed_ids = []
    success_count = 0
    
    # Cấu hình session
    timeout = ClientTimeout(total=30)
    conn = aiohttp.TCPConnector(limit=None)
    semaphore = asyncio.Semaphore(7)  
    
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        # Tạo list các coroutines
        tasks = [fetch_novel_info(session, id, semaphore) for id in ids]
        results = await asyncio.gather(*tasks)
        
        # Xử lý kết quả
        for id, html_content in results:
            if html_content:
                try:
                    novel_info = extract_info_from_html(id, html_content)
                    if novel_info:
                        processed_info = process_novel_info(novel_info)
                        if processed_info:
                            novel_data.append(processed_info)
                            print(f"Đã lấy thông tin truyện ID: {id}")
                            success_count += 1
                            continue
                except Exception as e:
                    print(f"Lỗi xử lý dữ liệu ID {id}: {str(e)}")
            
            failed_ids.append(id)
    
    return novel_data, failed_ids, success_count

In [126]:
# Hàm check driver 
def is_driver_alive(driver):
   """Kiểm tra trạng thái hoạt động của driver"""
   try:
       driver.title
       return True
   except:
       return False

In [127]:
# Hàm lấy thông tin truyện
def scrape_ids_and_info(driver, target_ids=None, start_id=None, end_id=None, filename='hako_data.csv'):
   novel_data = []
   failed_ids = []
   success_count = 0
   batch_size = 5
   delay_between_requests = 3
   delay_between_batches = 3
   
   if target_ids:
       ids_to_check = target_ids
       print(f"Kiểm tra {len(ids_to_check)} ID được chỉ định")
   else:
       missing_ids = get_missing_ids(start_id, end_id, filename)
       if not missing_ids:
           print(f"Không có ID nào thiếu trong khoảng từ {start_id} đến {end_id}")
           return []
       ids_to_check = missing_ids
       print(f"Tìm thấy {len(ids_to_check)} ID cần xử lý trong khoảng từ {start_id} đến {end_id}")
   
   try:
       for i in range(0, len(ids_to_check), batch_size):
           if not is_driver_alive(driver):
               print("\nChrome đã đóng, code đang dừng chạy...")
               raise Exception("Chrome đã đóng")
               
           batch_ids = ids_to_check[i:i + batch_size]
           batch_data = []
           
           print(f"\nXử lý batch {i//batch_size + 1}/{(len(ids_to_check) + batch_size - 1)//batch_size}")
           
           for id in batch_ids:
               try:
                   if not is_driver_alive(driver):
                       print("\nChrome đã đóng, code đang dừng chạy...")
                       raise Exception("Chrome đã đóng")
                       
                   url = f'https://ln.hako.vn/truyen/{id}-abcd'
                   driver.get(url)
                   time.sleep(delay_between_requests)
                   
                   if driver.current_url == url:
                       failed_ids.append(id)
                       continue
                       
                   html_content = driver.page_source
                   novel_info = extract_info_from_html(id, html_content)
                   
                   if novel_info:
                       processed_info = process_novel_info(novel_info)
                       if processed_info:
                           batch_data.append(processed_info)
                           print(f"Đã lấy thông tin truyện ID: {id}")
                           success_count += 1
                   else:
                       failed_ids.append(id)
               
               except Exception as e:
                   if "Chrome đã đóng" in str(e):
                       raise  
                   print(f"Lỗi khi xử lý ID {id}: {str(e)}")
                   failed_ids.append(id)
                   continue
           
           # Lưu dữ liệu của batch
           if batch_data:
               try:
                   save_to_csv(batch_data, filename, show_summary=False)
                   novel_data.extend(batch_data)
               except Exception as e:
                   print(f"Lỗi khi lưu batch data: {str(e)}")
                   # Thêm các ID trong batch vào failed_ids nếu lưu thất bại
                   failed_ids.extend([item['ID'] for item in batch_data])
           
           # Delay giữa các batch
           if i + batch_size < len(ids_to_check):
               print(f"Nghỉ {delay_between_batches} giây trước batch tiếp theo...")
               time.sleep(delay_between_batches)
               
   except KeyboardInterrupt:
       print("\nPhát hiện lệnh dừng từ người dùng. Đang lưu trạng thái hiện tại...")
       
   except Exception as e:
       if "Chrome đã đóng" in str(e):
           print("\nCode dừng chạy do Chrome đã đóng.")
       else:
           print(f"\nLỗi trong quá trình crawl: {str(e)}")
       
   finally:
       # Lưu ID thất bại
       if failed_ids:
           try:
               save_failed_ids(failed_ids)
           except Exception as e:
               print(f"Lỗi khi lưu failed IDs: {str(e)}")
       
       # Hiển thị thông báo tổng kết
       print(f"\nKết quả cuối cùng:")
       print(f"Tổng số bản ghi thành công: {success_count}")
       print(f"Tổng số bản ghi thất bại: {len(failed_ids)}")
       if failed_ids:
           print(f"Các ID thất bại đã được lưu vào file hako_null.csv")
   
   return novel_data

In [128]:
# Main
if __name__ == "__main__":
   username = "********"    # Điền username
   password = "********"    # Điền password
   
   while True:  
       try:
           driver = login_hako(username, password)
           
           if driver:
               choice = input("Chọn cách lấy ID (1: Nhập khoảng ID (từ... đến...), 2: Nhập list ID): ")
               
               if choice == "1":
                   start_id = int(input("Nhập ID bắt đầu: "))
                   end_id = int(input("Nhập ID kết thúc: "))
                   novel_data = scrape_ids_and_info(driver, start_id=start_id, end_id=end_id)
               else:
                   
                   input_ids = input("Nhập các ID (cách nhau bằng dấu phẩy): ").strip()
                   if input_ids:
                       target_ids = []
                       for id_str in input_ids.split(','):
                           id_str = id_str.strip()
                           if id_str.isdigit():  
                               target_ids.append(int(id_str))
                       
                       if target_ids:
                           novel_data = scrape_ids_and_info(driver, target_ids=target_ids)
                       else:
                           print("Không có ID hợp lệ")
                           continue
                   else:
                       print("Nhập ít nhất một ID:")
                       continue
               
               driver.quit()
               break  
           
       except Exception as e:
           print(f"Lỗi: {str(e)}")
           retry = input("Bạn có muốn thử lại không? (y/n): ")
           if retry.lower() != 'y':
               break

Đăng nhập thành công!
Kiểm tra 1 ID được chỉ định

Xử lý batch 1/1
Đã lấy thông tin truyện ID: 1

Kết quả cuối cùng:
Tổng số bản ghi thành công: 1
Tổng số bản ghi thất bại: 0
